Železniške povezave so podane v obliki slovarja. Ključi so imena postaj (npr. Zidani most), pripadajoče vrednosti pa seznami povezav na neposredno povezane postaje. Povezava je trojka z imenom ciljne postaje (npr. Laško), časom odhoda (v minutah od polnoči) s postaje v ključu (Zidani most) in časom prihoda na ciljno postajo (npr. Laško). Če ključu `"Zidani Most"` pripada seznam `[('Laško', 364, 382), ('Hrastnik', 365, 386), ('Laško', 375, 390), ('Radeče', 379, 395)]`, iz Zidanega Mostu odhaja vlak v Laško ob 6:04 in prispe tja ob 6:22, nato v Hrastnik ob 6:05 s prihodom ob 6:26, potem spet v Laško ob 6:15 s prihodom ob 6:30, potem v Radeče ob 6:19 s prihodom ob 6:35. **Povezave so urejene po časih odhodov.** Časi so izmišljeni in vsaka povezava z resničnimi prihodi in odhodi vlakov je zgolj naključna. V tem smislu so podobni dejanskim voznim redom Slovenskih železnic.

## 1. Postaje

Napišite funkcijo `postaje(povezave)`, ki prejme slovar s povezavami in vrne tri sezname: seznam križišč, seznam prehodnih postaj in seznam končnih. Postaja je križišče, če z nje vozijo vlaki na tri ali več različnih postaj. Postaja je prehodna, če je povezana z dvema različnima postajama, in končna, če vlaki z nje vozijo na eno samo postajo. Vse povezave so dvosmerne: če vozi vlak iz A v B, vozi tudi iz B v A.

### Rešitev

Začeli smo torej z nalogo iz slovarjev in seznamov ter, če ste se tako odločili množic.

Funkcija bo v vsakem primeru sestavljena iz dveh delov. V prvem bomo šli čez vse povezave in pripravili slovar, katerega ključi bodo povezave, vrednosti pa seznami (ali, v kasnejši, preprostejši različici) množice postaj, s katerimi je ta postaja povezana. V drugem delu bomo šli čez ta slovar in njegove ključe (imena postaj) metali v tri sezname (ali množice), glede na to, koliko elementov je v seznamu (množici), ki pripada temu ključu.

Najprej pokažimo najosnovnejšo - in najdaljšo - izvedbo te ideje.

In [2]:
def postaje(povezave):
    navezave = {}
    for postaja, odhodi in povezave.items():
        navezave[postaja] = []
        for ciljna, _, _ in odhodi:
            if ciljna not in navezave[postaja]:
                navezave[postaja].append(ciljna)

    koncne = []
    prehodne = []
    krizisca = []
    for postaja, povezane in navezave.items():
        if len(povezane) == 1:
            if postaja not in koncne:
                koncne.append(postaja)
        elif len(povezane) == 2:
            if postaja not in prehodne:
                prehodne.append(postaja)
        else:
            if postaja not in krizisca:
                krizisca.append(postaja)
    return krizisca, prehodne, koncne

`navezave` bodo torej slovar, katerega ključi bodo imena postaj, vrednosti pa seznami postaj, s katerimi je ta postaja povezana. Za vsako postajo iz `povezave`, gremo čez vsa imena postaj in odhode z nje (`for postaja, odhodi in povezave.items()`). Postajo dodamo v slovar. Znotraj zanke gremo čez vse odhode; zanimajo nas le imena postaja, na katere vozijo vlaki, ne pa tudi časi. Vsako postajo, ki je še ni v seznamu, dodamo vanj.

V drugem delu pripravimo tri sezname, v katere bomo dodajali imena postaj. Gremo čez slovar, ki smo ga pripravili v prvem delu in dodamo vsako postajo v ustrezni seznam - če je še ni tam.

Zanimivo je, da moramo tule gnezditi `if`-e. Če bi namesto

```
        if len(povezane) == 1:
            if postaja not in koncne:
                koncne.append(postaja)
```

pisali 

```
        if len(povezane) == 1 and postaja not in koncne:
            koncne.append(postaja)
```

bi se naslednji `else` (ali `elif`) preverjal tako v primeru, ko ni resničen pogoj `len(povezane) == 1` kot tudi takrat, ko je neresničen `postaja not in koncne`. Tako, kot smo (pravilno) sprogramirali zgoraj, pa se `else` oz. `elif` preverjata le, če ni resničen pogoj `len(povezane) == 1`.

Rešitev postane veliko preprostejša, če namesto seznamov povsod uporabimo množice. Ker množica lahko vsebuje vsak element le enkrat, se s tem znebimo vseh `if`-ov, ki preverjajo, ali je določena postaja že v seznamu. Ker naloga zahteva, da vrnemo sezname, na koncu (lahko kar v `return`-u, pretvorimo množice v sezname.

In [3]:
def postaje(povezave):
    navezave = {}
    for postaja, odhodi in povezave.items():
        navezave[postaja] = set()
        for ciljna, _, _ in odhodi:
            navezave[postaja].add(ciljna)
    koncne = set()
    prehodne = set()
    krizisca = set()
    for postaja, povezane in navezave.items():
        if len(povezane) == 1:
            koncne.add(postaja)
        elif len(povezane) == 2:
            prehodne.add(postaja)
        else:
            krizisca.add(postaja)
    return list(krizisca), list(prehodne), list(koncne)

Tudi `defaultdict` bi nam prihranil eno vrstico, `navezave[postaja] = set()`, hkrati pa dodal en `import`, zato tu nima posebnega smisla. Tu namreč točno vemo, kdaj moramo dodati postajo v slovar navezave: vedno, takoj znotraj zunanje zanke.

Kdor zna malo več, lahko naredi

In [4]:
def postaje(povezave):
    navezave = {}
    for postaja, odhodi in povezave.items():
        navezave[postaja] = {ciljna for ciljna, _, _ in odhodi}
    koncne = {k for k, v in navezave.items() if len(v) == 1}
    prehodne = {k for k, v in navezave.items() if len(v) == 2}
    krizisca = set(navezave) - prehodne - koncne
    return list(krizisca), list(prehodne), list(koncne)

Kdor zna to uporabiti še malo boljše, pa napiše celo

In [5]:
def postaje(povezave):
    navezave = {postaja: {ciljna for ciljna, _, _ in odhodi}
                for postaja, odhodi in povezave.items()}
    koncne = {k for k, v in navezave.items() if len(v) == 1}
    prehodne = {k for k, v in navezave.items() if len(v) == 2}
    krizisca = set(navezave) - prehodne - koncne
    return list(krizisca), list(prehodne), list(koncne)

## 2. Prihod

Napišite funkcijo `naslednja_povezava(povezave, odkod, kam, cas)`, ki poišče prvi vlak, ki odide iz odkod v kam ob podenam času cas ali po njem. Funkcije naj vrne čas, ko bo ta vlak prispel v kam. Če ob tem času (oziroma kasneje) ni več nobenega vlaka iz odkod v kam, ali pa takšne povezave sploh ni, funkcije vrne `None`. Klic `naslednja_povezava(povezave, "Zidani Most", "Laško", 362)` vrne `390`.

Napišite funkcijo `prihod(povezave, pot, zacetek)`, ki vrne čas, ob katerem bo potnik, ki prepotuje podano pot (seznam imen postaj), prispel na končno postajo poti, če se nanjo odpravi ob času zacetek. Potnik na vsaki postaji uporabi prvi možni vlak do naslednje postaje. Če poti ni možno opraviti, ker določena povezava ne obstaja ali pa na tisti dan ni več vlaka na tej povezavi, funkcije vrne None. Predpostavite, da vlaki Slovenskih železnic nimajo zamud.

Klic `prihod(povezave, ["Kranj", "Škofja Loka", "Ljubljana", "Litija", "Hrastnik"], 420)` vrne `585`. Potnik ob 439 sede na vlak Kranj – Škofja Loka, kamor prispe ob 464 in takoj nadaljuje v Ljubljano s prihodom ob 480. Do 542 čaka na vlak, ki ga ob 565 pripelje v Litijo, takoj nadaljuje v Hrastnik in prispe ob 585.

### Rešitev

Prva funkcija je preprosta: potrebno je iti prek odhodov s posamične postaje (za kar moramo le znati priti do vrednosti, ki pripada določenemu ključu v slovarju -- se pravi, znati moramo uporabiti podani slovar) in poiskati povezavo, ki vodi tja, kamor želimo, takrat, ko želimo (ali kasneje).

In [6]:
def naslednja_povezava(povezave, odkod, kam, cas):
    for k, odhod, prihod in povezave[odkod]:
        if k == kam and odhod >= cas:
            return prihod
    return None

Druga funkcija zahteva, da znamo narediti zanko čez seznam parov v slovarju -- nekaj, kar smo naredili na praktično vsakem predavanju. Za vsak par pokličemo prejšnjo funkcijo in sproti beležimo čas, ko se znajdemo na določeni povezavi. Če prejšnja funkcija vrne `None`, pa je veselja konec.

In [7]:
from itertools import pairwise

def prihod(povezave, pot, zacetek):
    cas = zacetek
    for odkod, kam in pairwise(pot):
        naslednji_cas = naslednja_povezava(povezave, odkod, kam, cas)
        if naslednji_cas is None:
            return None
        cas = naslednji_cas
    return cas

## 3. Vozni redi

Napišite funkcijo `vozni_red(povezave, postaja, ime_datoteke)`, ki v datoteko s podanim imenom izpiše odhode vlakov s podane postaje. Datoteka mora biti oblikovana, kot kaže spodnji izpis: imenu ciljne postaje sledijo odhodi z nje, nato prazna vrstica in naslednja ciljna postaja. Izpis kaže del odhodov s postaje Grosuplje.

```
Ivančna Gorica
06:43
07:39
08:44
09:41
10:31

Ribnica
05:58
07:00
08:03
08:56

Škofljica
07:43
08:29
09:42
10:42
```

### Rešitev

Tale naloga je malo podobna prvemu delu prve, le da gremo zdaj čez seznam, ki pripada želeni odhodni postaji in v seznam, katerega ključi so ciljne postaje, zlagamo odhodne čase.

Nato gremo prek tega slovarja in izpisujemo. Oblikovanja nizov ni veliko: znati moramo le izpisovati na dve mesti tako, da na začetek dodamo vodilno ničlo. No, pa celoštevilsko deljenje in ostanek po deljenju moramo znati poiskati.

In [12]:
def vozni_red(povezave, postaja, ime_dat):
    cilji = []
    for kam, odhod, _ in povezave[postaja]:
        if kam not in cilji:
            cilji.append(kam)
        cilji[kam].append(odhod)
        
    f = open(ime_dat, "wt", encoding="utf-8")
    for kam, odhodi in sorted(cilji.items()):
        f.write(kam + "\n")
        for odhod in odhodi:
            f.write(f"{odhod // 60:0>2}:{odhod % 60:0>2}\n")
        f.write("\n")

Tule bi pa lahko uporabili `defaultdict`. Vsaj jaz bi ga. Program pa bi se skrajšal ... no, za dve vrstici.

Detajl: `cilji.items()` vrne seznam parov (ključ, vrednost). `sorted` uredi pare po prvem elementu. Če bi bil prvi element enak, bi urejal po drugem, vendar se to tu tako ali tako ne more zgoditi, saj so ključi unikatni.

Še en detajl: med časi za posamične postaje je prazna vrsta, ki jo dobimo z `f.write("\n")`.

## 4. Rentabilnost

Naslednje funkcije bodo prejele dve numpyjevi tabeli: `postaje` je tabela z imeni postaj, `potniki[i, j]` pa vsebuje število potnikov, ki se je na določen dan prepeljalo s `postaje[i]` na `postaje[j]`. Napišite naslednje funkcije.

- `naj_odhodov(postaje, potniki)` vrne ime postaje, s katere se je odpeljalo največ potnikov.
- `naj_prihodov(postaje, potniki)` vrne ime postaje, na katero je prispelo največ potnikov.
- `prometnost(postaje, potniki, n)` vrne tabelo z imeni `n` postaj z največjim skupnim številom prihodov in odhodov. Tabela naj bo urejena padajoče; najbolj prometna postaja naj bo na prvem mestu.

### Rešitev

Tole je bila naloga iz osi v numpyju in iz kombiniranja argmax z indeksiranjem. Prvi dve funkciji sta bili res kratki.

In [9]:
def naj_odhodov(postaje, potniki):
    return postaje[np.argmax(np.sum(potniki, axis=1))]

def naj_prihodov(postaje, potniki):
    return postaje[np.argmax(np.sum(potniki, axis=0))]

Za drugo pa je bilo potrebno sešteti, kar smo računali v prejšnjih dveh funkcijah, z argsort dobiti urejene indekse, jih obrniti (da jih uredimo padajoče) in vzeti prvih `n`.

In [10]:
def prometnost(imena_postaj, potniki, k):
    promet = np.sum(potniki, axis=1) + np.sum(potniki, axis=0)
    indeks = np.argsort(promet)[-k:][::-1]
    return imena_postaj[indeks]

Seveda gre tudi to v eni vrstici, vendar bodimo raje pregledni kot kratki.

## 5. Vozovnice

Ko boste prvič pognali teste, se bo pojavil direktorij z imenom *vozovnice*, v katerem bodo shranjeni podatki o vozovnicah nekaj potnikov. Obliko datotek prepoznajte sami.

Napišite funkcijo `preberi_vozovnice(direktorij)`, ki prejme ime direktorija in vrne slovar, katerega ključi so imena potnikov, pripadajoče vrednosti pa slovar z vsemi drugimi podatki, zapisanimi v datoteki. SŽ predpostavljajo, da se imena potnikov ne ponavljajo.

Pomoč: funkcija `os.listdir(dir)` vrne imena vseh datotek v podanem direktoriju. Funkcija, ki naloži podatke v formatu "XYZ", se navadno nahaja v modulu "XYZ" in se imenuje `load`, kot argument pa prejme (odprto) datoteko, iz katere mora naložiti podatke.

### Rešitev

Končnica datotek je .json in že iz tega lahko uganemo, da je format json. Potrebujemo torej funkcijo `json.load`.

Primerna rešitev, je, recimo

In [11]:
def preberi_vozovnice(direktorij):
    vozovnice = {}
    for ime_dat in os.listdir(direktorij):
        podatki = json.load(open(os.path.join(direktorij, ime_dat), "r", encoding="utf-8"))
        vozovnice[podatki.pop("Ime")] = podatki
    return vozovnice

Funkcija `os.path.join(direktorij, ime_dat)` vrne `direktorij + "\\" + ime_dat` na Windowsih in `direktorij + "/" + ime_dat` na vseh drugih operacijskih sistemih. Če niste vedeli zanjo - nič hudega, direktorije ste lahko združevali sami. Nekaj podobnega ste počeli v domači nalogi s skrivnim sporočilom.

Bolj zanimiv je `podatki.pop("Ime")`. Ta klic vrne vrednost, ki pripada ključu `"Ime"` in ta ključ pobriše iz slovarja. Če ne vemo zanjo (in najbrž je res nismo posebej omenjali), morate brisanje in pobiranje imena pač ločiti. Torej:

In [13]:
def preberi_vozovnice(direktorij):
    vozovnice = {}
    for ime_dat in os.listdir(direktorij):
        podatki = json.load(open(os.path.join(direktorij, ime_dat), "r", encoding="utf-8"))
        ime = podatki["Ime"]
        del podatki["Ime"]
        vozovnice[ime] = podatki
    return vozovnice

Če smo v stiski in se ne spomnimo niti na `del`, pa bomo (kar nam sicer ne bo v čast) sestavili nov slovar, v katerega bomo prepisali vsebino slovarja podatki, razen ključa `"Ime"`.